## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key = g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Hithadhoo,MV,-0.6000,73.0833,82.38,73,100,10.38,overcast clouds,2021-08-07 20:18:24
1,1,Nishihara,JP,26.1842,127.7558,78.85,97,90,19.57,light intensity shower rain,2021-08-07 20:18:24
2,2,Hobart,AU,-42.8794,147.3294,35.26,89,75,3.00,broken clouds,2021-08-07 20:18:24
3,3,Khatanga,RU,71.9667,102.5000,50.22,68,73,8.37,broken clouds,2021-08-07 20:18:24
4,4,Monki,PL,53.4050,22.7979,59.70,88,33,6.44,scattered clouds,2021-08-07 20:18:25


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Hithadhoo,MV,-0.6000,73.0833,82.38,73,100,10.38,overcast clouds,2021-08-07 20:18:24
1,1,Nishihara,JP,26.1842,127.7558,78.85,97,90,19.57,light intensity shower rain,2021-08-07 20:18:24
5,5,Hasaki,JP,35.7333,140.8333,79.20,88,100,14.63,heavy intensity rain,2021-08-07 20:13:40
6,6,Whitehorse,CA,60.7161,-135.0538,63.10,82,75,4.61,broken clouds,2021-08-07 20:17:11
7,7,Albany,US,42.6001,-73.9662,87.15,67,99,1.99,overcast clouds,2021-08-07 20:15:43
8,8,Guadix,ES,37.2993,-3.1392,78.46,26,0,4.59,clear sky,2021-08-07 20:18:25
9,9,Komsomolskiy,UZ,40.4272,71.7189,79.52,36,65,11.50,broken clouds,2021-08-07 20:18:26
10,10,Sao Filipe,CV,14.8961,-24.4956,79.20,80,47,5.97,scattered clouds,2021-08-07 20:18:26
11,11,Pemberton,CA,50.3164,-122.8027,65.89,92,100,4.25,overcast clouds,2021-08-07 20:18:26
12,12,Sapouy,BF,11.5544,-1.7736,79.21,72,96,9.15,overcast clouds,2021-08-07 20:18:26


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                455
City                   455
Country                454
Lat                    455
Lng                    455
Max Temp               455
Humidity               455
Cloudiness             455
Wind Speed             455
Current Description    455
Date                   455
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                454
City                   454
Country                454
Lat                    454
Lng                    454
Max Temp               454
Humidity               454
Cloudiness             454
Wind Speed             454
Current Description    454
Date                   454
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hithadhoo,MV,82.38,overcast clouds,-0.6000,73.0833,
1,Nishihara,JP,78.85,light intensity shower rain,26.1842,127.7558,
5,Hasaki,JP,79.20,heavy intensity rain,35.7333,140.8333,
6,Whitehorse,CA,63.10,broken clouds,60.7161,-135.0538,
7,Albany,US,87.15,overcast clouds,42.6001,-73.9662,
8,Guadix,ES,78.46,clear sky,37.2993,-3.1392,
9,Komsomolskiy,UZ,79.52,broken clouds,40.4272,71.7189,
10,Sao Filipe,CV,79.20,scattered clouds,14.8961,-24.4956,
11,Pemberton,CA,65.89,overcast clouds,50.3164,-122.8027,
12,Sapouy,BF,79.21,overcast clouds,11.5544,-1.7736,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# # 6b. Iterate through the hotel DataFrame.
for index, hotel in hotel_df.iterrows():
    
     # 6c. Get latitude and longitude from DataFrame
    
    lat = hotel["Lat"]
    lng = hotel["Lng"]
    params["location"]= f'{lat},{lng}'
    
     # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels = requests.get(base_url, params=params).json()
    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   454
Country                454
Max Temp               454
Current Description    454
Lat                    454
Lng                    454
Hotel Name             454
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current description</dt><dd>{Current Description}</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))